In [6]:
#! Install library
import os
import importlib.util

if importlib.util.find_spec("requests") is None:
    os.system("pip install requests")

if importlib.util.find_spec("beautifulsoup4") is None:
    os.system("pip install beautifulsoup4")

if importlib.util.find_spec("Counter") is None:
    os.system("pip install Counter")

if importlib.util.find_spec("openpyxl") is None:
    os.system("pip install openpyxl")

In [7]:
Debug = False

In [8]:
#! Authentication details
import os
import dotenv
import importlib.util

JiraHost = None
ConfluenceHost = None
Username = None
Password = None

if importlib.util.find_spec("google.colab") is not None:  ## if using google colab
    if not os.path.exists(".env"):
        from google.colab import files

        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, ".env")
        except:
            pass

try:
    dotenv.load_dotenv("../.env", override=True)

    JiraHost = os.getenv("SECRETS_HOST")
    ConfluenceHost = os.getenv("SECRETS_CONFLUENCE")
    Username = os.getenv("SECRETS_USERNAME")
    Password = os.getenv("SECRETS_PASSWORD")
except:
    display("trouble loading dot env")
    pass

if JiraHost is None or JiraHost == "":
    JiraHost = input("Enter Jira Host")

if ConfluenceHost is None or ConfluenceHost == "":
    ConfluenceHost = input("Enter Confluence Host")

if Username is None or Username == "":
    Username = input("Enter Username")

if Password is None or Password == "":
    Password = input("Enter Password")

display("Jira Host: " + JiraHost)
display("Confluence Host: " + ConfluenceHost)

'Jira Host: https://jiradev.budgetdirect.com.au/'

'Confluence Host: https://confluencedev.budgetdirect.com.au/'

In [9]:
#! Functions
import base64
import re
from bs4 import BeautifulSoup
import requests
from collections import Counter
import pandas as pd
import requests
from functools import reduce
import warnings
import json

requests.packages.urllib3.disable_warnings(
    requests.packages.urllib3.exceptions.InsecureRequestWarning
)


def _ExpandColumn(
    self: pd.DataFrame,
    colName: str,
    columnsToExpand=[],
    prefix: str = "Prefix",
    sentenceCase: bool = True,
) -> pd.DataFrame:
    if prefix == "Prefix":
        prefix = colName + " "
        with warnings.catch_warnings():
            warnings.simplefilter(action="ignore", category=FutureWarning)
            expandedCols = self[colName].apply(
                lambda x: pd.Series(x).add_prefix(prefix)
            )
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))

    if len(columnsToExpand) > 0:
        expandedCols = expandedCols[columnsToExpand]

    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns]

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)


pd.DataFrame.expand = _ExpandColumn


def fnSentenceCase(s):
    s = " ".join(dict.fromkeys(s.split()))  # remove duplicate words
    s = s.replace("0", "")  # remove "0"
    s = s.strip()
    return " ".join(
        [x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]
    )  # sentence case


def _SentenceCaseColumns(self: pd.DataFrame) -> pd.DataFrame:
    self.columns = [fnSentenceCase(c) for c in self.columns]
    return self


pd.DataFrame.sentence_case_columns = _SentenceCaseColumns


def flatten_reduce_lambda(frm):
    try:
        return list(reduce(lambda x, y: x + y, frm, []))
    except:
        return list(reduce(lambda x, y: x + y, [frm], []))

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic "
        + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120",
    }


def fnUsers():
    def ApiCall(startAt):
        url = "/rest/api/2/user/list?limit=1000&startAt=" + str(startAt)
        headers = fnGetDefaultHeaders()
        response = requests.get(ConfluenceHost + url, headers=headers, verify=False)
        return response.json()

    df = pd.DataFrame(flatten_reduce_lambda([ApiCall(i * 1000) for i in range(8)]))
    df = df.dropna(how="all")
    return df


def fnAPI(webRequestDelegate, startAt=0) -> pd.DataFrame:
    def innerGetResults(webRequestDelegate, startAt=0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(
                    webRequestDelegate, startAt + results["maxResults"]
                )
            else:
                return [results]
        else:
            return [results]

    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df

In [10]:
users = fnUsers()

display(users.head())

ConnectionError: HTTPSConnectionPool(host='confluencedev.budgetdirect.com.au', port=443): Max retries exceeded with url: //rest/api/2/user/list?limit=1000&startAt=0 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000195AAE93C10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))